In [39]:
!pip install cvxpy
import pandas as pd
import numpy as np
import cvxpy as cp


In [40]:
df = pd.read_csv("aoe4_data.csv")
print(list(df.columns))
df = df.dropna()


[' ', 'Unnamed: 1', 'Unnamed: 2', 'Resource Cost', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Time', 'Pop', 'Move', 'HP', 'HP per\nResource', 'Armor', 'Unnamed: 13', 'Attack', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Bonus', 'Note']


In [41]:

df.rename(columns={"Resource Cost": "Food", "Unnamed: 4": "Wood","Unnamed: 5": "Gold",}, inplace=True)
df.rename(columns={"Unnamed: 15" :"Spd","Unnamed: 16": "DPS","Unnamed: 17": "Rng"},inplace =True)
df.drop(columns=["Unnamed: 1", "Unnamed: 2","Bonus","Note","Unnamed: 19","Unnamed: 18","Unnamed: 13","Unnamed: 6","Move", "DPS", "HP per\nResource"], inplace=True)
df = df.iloc[:124]
df = df.dropna(subset=['Food', 'Gold', 'Wood'])

In [42]:
df.head()
df = df.dropna()
# 100 wood is 130 gold
# 100 food is 130 gold

In [43]:
df.Food *= 200
df.Wood *= 200
df.Gold = df.Food + df.Wood + df.Gold
df.drop(columns=["Food", "Wood"], inplace=True)
df.head()

,,Gold,Time,Pop,HP,Armor,Attack,Spd,Rng
2,Spearman,16000.0,15.0,1.0,70.0,0,6,1.75,0.29
19,Longbowman,18000.0,15.0,1.0,70.0,0,6,1.5,7.00
29,Arbalétrier,16040.0,22.0,1.0,80.0,1,12,2,5.00
34,Scout,14000.0,20.0,1.0,110.0,0,2,2,0.29
40,Camel Rider,36060.0,35.0,1.0,270.0,0,14,1,0.42


In [44]:
# Time, HP, Pop, Armor, Attack, Speed and Range would be our constraints
df
print(len(df.index))


25


In [45]:
c = np.array(df['Gold'])
A = np.array(df[['Time','HP','Armor','Attack', "Spd", "Rng"]]).T
P = np.array(df["Pop"]).T
df["Armor"] = df["Armor"].astype(float)
df["Attack"] = df["Attack"].astype(float)
df["Spd"] = df["Spd"].astype(float)
df["Rng"] = df["Rng"].astype(float)
b = np.array([1026,5000,20,1000,100,130]) # These are our requirements

In [73]:
X = cp.Variable(df.shape[0])

constraints = [
    A @ X >= b, 
    X >= 0,  
    P @ X <= 200  
]

for i in range(df.shape[0]):
    constraints.append(X[i] <= 0.35 * cp.sum(X))  #No single unit type can exceed 50% of the total population

elite_units = cp.Variable(df.shape[0], boolean=True)
constraints.append(cp.sum(elite_units) >= 0.1 * cp.sum(X))
for i in range(df.shape[0]):
    constraints.append(elite_units[i] * 600 <= c[i])  # Only count units costing >600 as elite

obj = cp.Minimize(c @ X)
prob = cp.Problem(obj, constraints)
prob.solve()


845791.9343687746

In [74]:
for i in range(df.shape[0]):
    unit_name = df.iloc[i, 0] 
    if X.value[i] > 0:
        print(df.iloc[i][' '],abs(X.value.round(1)[i]))

Longbowman 3.9
Arbalétrier 14.4
Scout 14.4
War Elephant 1.4
Dhow 6.4
Explosive Dhow 0.4
Lodya Demolition Ship 0.2
